%

In [1]:
import numpy as np

In [2]:
import torchvision
import torchvision.datasets

In [3]:
import torch
import torch.nn

simple toy model to have something to work with

In [4]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(1, 16, kernel_size=3, padding=0),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=3, stride=2),
    torch.nn.Flatten(),
    torch.nn.Linear(16 * 3 * 3, 10)
)

load the MNIST dataset<br>
this comes with all torch niceties and helpers, but we will try to get the<br>
raw data from here (not all the way to the file system, but close enough)

In [5]:
dataset = torchvision.datasets.MNIST(
    root='../data/mnist',
    train=True,
    download=True
)

here we extract the data from the dataset and we store it as a simple array of numbers<br>
DO NOT DO THIS, this is just for explanation purposes

here we create some simple examples of preprocessing operations

In [23]:
transform_np_image_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((10, 10)),
    torchvision.transforms.ToTensor()
])
transform_np_vector_to_tensor = torch.tensor

In [24]:
from sklearn.model_selection import train_test_split

# Convert to NumPy arrays
images = np.array(dataset.data)
labels = np.array(dataset.targets)

# Split into train/val (e.g., 80% train, 20% val)
train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(images, labels, test_size=0.2)

# Zip them into datasets
dataset_numpy = list(zip(train_imgs, train_lbls))
validation_dataset = list(zip(val_imgs, val_lbls))

In [25]:
def training_loop(num_epoch, model, dataset, validation_dataset, loss_f, optimizer):
    for epoch in range(num_epoch):
        model.train()
        epoch_loss = 0

        for images, labels in dataset:
            x = transform_np_image_to_tensor(images).unsqueeze(0)
            y = transform_np_vector_to_tensor(labels).unsqueeze(0)

            output = model(x)
            loss = loss_f(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Validation
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for val_images, val_labels in validation_dataset:
                x_val = transform_np_image_to_tensor(val_images).unsqueeze(0)
                y_val = transform_np_vector_to_tensor(val_labels).unsqueeze(0)

                out_val = model(x_val)
                loss_val = loss_f(out_val, y_val)
                epoch_val_loss += loss_val.item()

        print(f"Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}")
        print(f"Epoch {epoch+1}: Validation Loss = {epoch_val_loss:.4f}")


In [26]:
training_loop(
    10,
    model,
    dataset_numpy,
    validation_dataset,
    loss_f = torch.nn.L1Loss(),
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
)

/opt/anaconda3/envs/ml-env/lib/python3.11/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1: Train Loss = 66652.5946
Epoch 1: Validation Loss = 15463.4850
Epoch 2: Train Loss = 63471.1809
Epoch 2: Validation Loss = 15814.3972
Epoch 3: Train Loss = 63175.6020
Epoch 3: Validation Loss = 15308.9369
Epoch 4: Train Loss = 62932.9859
Epoch 4: Validation Loss = 14722.3378
Epoch 5: Train Loss = 63086.6762
Epoch 5: Validation Loss = 15417.6743
Epoch 6: Train Loss = 62894.6328
Epoch 6: Validation Loss = 14823.7376
Epoch 7: Train Loss = 62858.3646
Epoch 7: Validation Loss = 15006.8733
Epoch 8: Train Loss = 62846.8524
Epoch 8: Validation Loss = 15326.1170
Epoch 9: Train Loss = 63055.3898
Epoch 9: Validation Loss = 15204.2783
Epoch 10: Train Loss = 62850.0959
Epoch 10: Validation Loss = 15528.4370


%<br>
additional exercise (if this is of interest to you)<br>
based of the CelebA_Local dataset provided in the solution for exercise 4,<br>
can you build a data pipeline yourself using torch datasets?<br>
try to use it to do some of the tasks outlined in exercise 5